# Test conversion from True to Applied wind

## Definition of function

In [14]:
import numpy as np

def tw_2_aw(twa, tws, boat_speed):
    """
    Calculate the Apparent Wind Angle (AWA) for a given TWA, TWS, and boat speed.
    Formulas taken from:
    https://orc.org/uploads/files/Rules-Regulations/2025/Speed-Guide-Explanation-2025.pdf
    Args:
        twa (float): True Wind Angle in degrees
        tws (float): True Wind Speed in knots
        boat_speed (float): Boat speed in knots
        mode: 'beat' or 'run'
        
    Returns:
        float: Apparent Wind Angle in degrees
    """

    # Convert angles to radians for numpy
    btw = np.radians(twa) # bearing of true wind
    baw = np.arctan((tws*np.sin(btw))/(tws*np.cos(btw) + boat_speed))
        
    # Calculate AWS
    # vaw = np.sqrt(tws**2 + boat_speed**2 - 2 * tws * boat_speed * np.cos(phi_rad))
    vaw = np.sqrt((tws*np.sin(btw))**2 + ((tws*np.cos(btw) + boat_speed)**2))

    # Calculate AWA
    #baw = np.arccos(-1*(tws**2 - aws**2 - boat_speed**2) / (2 * tws * aws)) 
    baw = np.arctan((tws*np.sin(btw))/(tws*np.cos(btw) + boat_speed))
    awa = np.degrees(baw)
    if awa < 0:
        print("DEBUG: Applied wind angle is negative, adding 180 degrees")
        awa = awa + 180
    
    return {'awa': round(awa,2), 'aws': round(vaw,2)}

## Test the function

In [15]:
tws = 20
twa = 140
boat_speed = 10

awa = tw_2_aw(twa, tws, boat_speed)['awa']
aws = tw_2_aw(twa, tws, boat_speed)['aws']

print(f"AWS: {aws} kn | AWA: {awa}°")

DEBUG: Applied wind angle is negative, adding 180 degrees
DEBUG: Applied wind angle is negative, adding 180 degrees
AWS: 13.91 kn | AWA: 112.48°


# GPS position readout from GPS Mouse

## GPSD reader class

In [1]:
import serial
import pynmea2
import time
from typing import Optional, Tuple

class GPSSpeedReader:
    def __init__(self, port: str = '/dev/ttyUSB0', baud_rate: int = 4800):
        """
        Initialize GPS reader with specified port and baud rate.
        
        Args:
            port: Serial port where GPS device is connected (default: /dev/ttyUSB0)
            baud_rate: Communication speed (default: 4800, common for GPS devices)
        """
        self.port = port
        self.baud_rate = baud_rate
        self.serial_connection = None
        
    def connect(self) -> bool:
        """
        Establish connection to the GPS device.
        
        Returns:
            bool: True if connection successful, False otherwise
        """
        try:
            self.serial_connection = serial.Serial(
                port=self.port,
                baudrate=self.baud_rate,
                timeout=1  # 1 second timeout for reading
            )
            return True
        except serial.SerialException as e:
            print(f"Error connecting to GPS device: {e}")
            return False
            
    def read_speed(self) -> Optional[Tuple[float, float]]:
        """
        Read and parse NMEA data to get current speed.
        
        Returns:
            Optional[Tuple[float, float]]: Tuple of (speed_knots, speed_kph) or None if no valid data
        """
        if not self.serial_connection or not self.serial_connection.is_open:
            return None
            
        try:
            # Read lines until we get a valid NMEA sentence
            while True:
                line = self.serial_connection.readline().decode('ascii', errors='replace')
                
                # Check if we have a valid NMEA sentence
                if line.startswith('$'):
                    msg = pynmea2.parse(line)
                    
                    # RMC (Recommended Minimum Navigation Information) contains speed
                    if isinstance(msg, pynmea2.RMC):
                        if msg.spd_over_grnd is not None:
                            speed_knots = float(msg.spd_over_grnd)
                            speed_kph = speed_knots * 1.852  # Convert knots to km/h
                            return (speed_knots, speed_kph)
                            
                    # VTG (Track Made Good and Ground Speed) also contains speed
                    elif isinstance(msg, pynmea2.VTG):
                        if msg.spd_over_grnd_kts is not None:
                            speed_knots = float(msg.spd_over_grnd_kts)
                            speed_kph = speed_knots * 1.852
                            return (speed_knots, speed_kph)
                            
        except (serial.SerialException, pynmea2.ParseError, ValueError) as e:
            print(f"Error reading GPS data: {e}")
            return None
            
    def close(self):
        """Clean up serial connection."""
        if self.serial_connection and self.serial_connection.is_open:
            self.serial_connection.close()

def main():
    """Example usage of the GPSSpeedReader class."""
    gps_reader = GPSSpeedReader()
    
    if not gps_reader.connect():
        print("Failed to connect to GPS device")
        return
        
    try:
        print("Reading GPS speed data. Press Ctrl+C to stop...")
        while True:
            speed_data = gps_reader.read_speed()
            if speed_data:
                speed_knots, speed_kph = speed_data
                print(f"Current speed: {speed_knots:.1f} knots ({speed_kph:.1f} km/h)")
            time.sleep(1)  # Read speed every second
            
    except KeyboardInterrupt:
        print("\nStopping GPS reading...")
    finally:
        gps_reader.close()

if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'serial'

In [1]:
import threading
import time
import math
import json
import websocket
import requests
from urllib.parse import urljoin

## SignalK Data retrieval

Forget about SignalK - it is too complicated, because OpenCPN is not able to output data via SignalK. It is also overkill, because OpenCPN can natively stream NMEA data to an UDP datastream.
It is possible to use both, NMEA0183 and NMEA200.

Next step: Find out what is easier, and then program a class which can read the data from a network stream.

ImportError: cannot import name 'SignalKReader' from 'signalK_reader' (/home/christian/Dropbox/scripts/python/polar_performance/signalK_reader.py)

In [3]:
from signalK_reader import SignalKReader
import time

# Example usage:

signalk_reader = SignalKReader()
try:
    while True:
        time.sleep(5)
        success, speed = signalk_reader.read_gps_speed()
        wind_success, tws, twa, twd = signalk_reader.read_wind_data()
        
        if success:
            print(f"Current speed: {speed:.2f} knots")
        else:
            print("No valid speed reading yet.")
            
        if wind_success:
            print(f"Wind data - TWS: {tws:.2f}kt, TWA: {twa:.1f}°, TWD: {twd:.1f}°")
        else:
            print("No valid wind data yet.")
            
except KeyboardInterrupt:
    print("Exiting...")
finally:
    signalk_reader.close()


**ERR SIGNALK: Error discovering endpoints: HTTPConnectionPool(host='localhost', port=3000): Max retries exceeded with url: /signalk (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f9d2cdd4910>: Failed to establish a new connection: [Errno 111] Connection refused'))
**ERR SIGNALK: No WebSocket URL available
**ERR SIGNALK: No WebSocket URL available
No valid speed reading yet.
No valid wind data yet.
**ERR SIGNALK: No WebSocket URL available
No valid speed reading yet.
No valid wind data yet.
**ERR SIGNALK: No WebSocket URL available
No valid speed reading yet.
No valid wind data yet.
**ERR SIGNALK: No WebSocket URL available
No valid speed reading yet.
No valid wind data yet.
**ERR SIGNALK: No WebSocket URL available
No valid speed reading yet.
No valid wind data yet.
**ERR SIGNALK: No WebSocket URL available
No valid speed reading yet.
No valid wind data yet.
**ERR SIGNALK: No WebSocket URL available
No valid speed reading yet.
No valid wind data yet.
**

# UDP data retrieval

In [1]:
from udp_gps_reader import UDPNetworkReader

import socket
import threading
import time
import math
import re

# Example Usage:
# 1. Configure OpenCPN:
#    - Go to Settings (wrench icon) -> Connections.
#    - Add a new connection.
#    - Type: Network. Protocol: UDP (or TCP). Address: (e.g., 0.0.0.0 or specific IP if sending from OpenCPN).
#    - Port: 10110 (or your chosen port).
#    - Direction: Output. Output on this port/address.
#    - NMEA Sentences: Select sentences to output (e.g., RMC, GGA, VTG, MWD, MWV).
#    - Apply and OK.

# This script will act as a UDP server listening on 0.0.0.0:10110
# Or as a TCP client connecting to OpenCPN if OpenCPN is TCP server.
# For TCP server mode in this script, _init_socket and _poll_data_stream would need changes.

# UDP Example (this script is a server)
print("Starting OpenCPN Network Reader (UDP Server Mode)...")
# Use '127.0.0.1' if OpenCPN is on the same machine and outputting to 127.0.0.1
# Use '0.0.0.0' to listen on all interfaces if OpenCPN outputs to this machine's IP or broadcast.
udp_reader = UDPNetworkReader(host='127.0.0.1', port=10110, connection_type='udp')

# TCP Client Example (this script is a client, OpenCPN is server)
# print("Starting OpenCPN Network Reader (TCP Client Mode)...")
# ocpn_reader = OpenCPNNetworkReader(host='localhost', port=10110, connection_type='tcp')

if udp_reader.sock is None:
    print("Failed to start reader. Exiting.")
    exit()

try:
    last_print_time = 0
    while True:
        time.sleep(1) # Main loop processing interval

        # Get all data at once
        current_data = udp_reader.get_latest_data()
        
        # Print info every 5 seconds if data has been updated
        if current_data['last_update_time'] > 0 and time.time() - last_print_time > 5:
            udp_reader._print_info() # Use the internal print method for formatted output
            last_print_time = time.time()

        # Example of getting specific data points:
        # success_sog, sog = ocpn_reader.get_sog_knots()
        # if success_sog:
        #     print(f"SOG: {sog:.2f} knots")
        # else:
        #     print("SOG: No valid SOG yet.")

        # success_wind, tws, twd, twa = ocpn_reader.get_true_wind()
        # if success_wind:
        #    print(f"Wind: TWS {tws:.1f}kn, TWD {twd:.0f}°, TWA {twa:.0f}°")

except KeyboardInterrupt:
    print("\nExiting due to KeyboardInterrupt...")
finally:
    udp_reader.close()
    print("Program terminated.")


Starting OpenCPN Network Reader (UDP Server Mode)...
**INFO NET: UDP listener started on 127.0.0.1:10110
21:40:51 **NET DATA: Fix: GPS, Sats: 11, HDOP: 1.3, Pos: 47°04.243'N, 15°26.371'N
             SOG: 5.1kn, COG: 45°
             TWS: 10.0kn, TWD: 46°, TWA: 1°
21:40:56 **NET DATA: Fix: GPS, Sats: 8, HDOP: 1.8, Pos: 47°04.248'N, 15°26.379'N
             SOG: 4.9kn, COG: 47°
             TWS: 10.2kn, TWD: 44°, TWA: -3°
21:41:01 **NET DATA: Fix: GPS, Sats: 9, HDOP: 1.5, Pos: 47°04.252'N, 15°26.386'N
             SOG: 5.0kn, COG: 47°
             TWS: 10.2kn, TWD: 38°, TWA: -10°
21:41:06 **NET DATA: Fix: GPS, Sats: 11, HDOP: 1.0, Pos: 47°04.257'N, 15°26.394'N
             SOG: 5.2kn, COG: 46°
             TWS: 10.9kn, TWD: 36°, TWA: -11°
21:41:11 **NET DATA: Fix: GPS, Sats: 8, HDOP: 1.1, Pos: 47°04.262'N, 15°26.401'N
             SOG: 5.1kn, COG: 43°
             TWS: 10.6kn, TWD: 34°, TWA: -9°
21:41:16 **NET DATA: Fix: GPS, Sats: 11, HDOP: 1.4, Pos: 47°04.267'N, 15°26.408'N
          

In [2]:
udp_reader.close()
print("UDP reader closed.")


**INFO NET: Closing network reader...
**INFO NET: UDP socket closed.
**INFO NET: Network reader closed.
UDP reader closed.


In [3]:
current_data

{'sog_knots': 4.6,
 'lat': 47.07129,
 'lon': 15.440376666666669,
 'cog_degrees': 43.7,
 'fix_mode': 1,
 'hdop': 1.9,
 'satellites_count': 9,
 'tws_knots': 10.1,
 'twd_degrees': 37.9,
 'twa_degrees': -5.800000000000004,
 'last_update_time': 1747683687.2710834}